<div style="font-size: 200%; font-weight: bold; color: maroon;">Spark Streaming: DStream, socket</div>
<!-- v 1.3 -- Paulo Villegas, 2021 -->

Un ejemplo sencillo de Spark Streaming clásico (RDD), leyendo datos desde un socket. Para poder mostrarlo de forma continua sobre el notebook usaremos _widgets_ de Jupyter

In [ ]:
import sys
import datetime

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.storagelevel import StorageLevel

Vamos a asegurarnos de que la configuración de Spark es la que necesitamos para poder ejecutar la aplicación de streaming en el _notebook_. 

En particular, nos hace falta que el servidor de Spark puede arrancar <span style="color: red;">al menos dos hebras</span> de ejecución. Si no las tiene, necesitamos modificar la configuración de Spark

In [ ]:
# Check that we have a valid Spark master config. In particular, we need at least 2 executor threads
cfg = sc.getConf()
cfg.get('spark.master')

In [ ]:
# Create a couple of Jupyter widgets to receive the streaming output
from ipywidgets import widgets

wgt = { 'label' : widgets.Label( layout=widgets.Layout(width='20em'),), 
        'text' :  widgets.HTML(layout=widgets.Layout(width='90%', height='15em', border='1px dashed gray')),
      }
output_widget = widgets.VBox( [wgt['label'],wgt['text']] )

In [ ]:
# Create the call that will receive the DStream RDD and write the data to the widget

from io import StringIO

def send2widget(label, text):
    """
    Send data to the widget
    """
    # Update the label widget
    wgt['label'].value = label
    # Compose the HTML buffer
    buf = StringIO()
    buf.write( u'<div style="white-space: pre-wrap; background-color: cornsilk; color: indigo;">' )
    for line in text:
        buf.write( line + '<br>' )
    buf.write( u'</div>')
    # Write it to the textarea widget
    wgt['text'].value = buf.getvalue()
    buf.close()

def output2widget(rdd):    
    """
    Get the RDD with the lines received in this batch and print them out
    """
    # Fetch from the cluster
    received = rdd.collect()
    # Send to the widget
    label = 'lines={:<3}   ts={}'.format(len(received), datetime.datetime.now())
    send2widget(label, received)


In [ ]:
# Define streaming configuration
wait = 2
host = 'localhost'
port = 9998

In [ ]:
# Create the Streaming context
ssc = StreamingContext(sc, wait)

# Input DStream: read lines from a socket
lines = ssc.socketTextStream( host, port, storageLevel=StorageLevel.MEMORY_ONLY )

# Send them for printing
lines.foreachRDD( output2widget )

In [ ]:
# Show the widget
output_widget

In [ ]:
# Start the loop
try:
    ssc.start()
    ssc.awaitTermination()
except KeyboardInterrupt:
    ssc.stop( stopSparkContext=False, stopGraceFully=True )
    wgt['label'].value = '<terminated>'
